In [1]:
# Importing required packages
import torch
import torch.nn as nn
from torch.autograd import Variable

from tqdm import tqdm
import os
import argparse

import string
import random
import time
import math

import numpy as np
import pandas as pd
from nltk.util import ngrams
#from google.colab import drive
from os import listdir
from os.path import isfile, join
import json
import time
import nltk
import re
from progressbar import ProgressBar
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import regexp_tokenize
import nltk.data 
import nltk
import matplotlib.pyplot as plt
import pickle
import networkx as nx
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import gensim
from gensim.models import word2vec
import logging
from scipy import stats
from scipy.spatial import distance
from fastdist import fastdist
from tsai.all import *


# download the nltk packages required
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

C:\Users\surta\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\surta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\surta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Extracting the data from the json files and storing in drive



In [2]:
path = 'C:/Users/surta/Downloads/pdf_json'
start = time.time()
filenames = [f for f in listdir(path) if isfile(join(path, f))]
end = time.time()

In [3]:
# Displaying the total number of json files in the shared drive folder
print("The total number of files found: ", len(filenames))

The total number of files found:  56528


In [ ]:
def extract_text(text):
    title_meta = []
    title_meta.append(text['metadata']['title'])
    text_abstract = [text['abstract'][entry]['text'] for entry in range(len(text['abstract']))]
    text_bodytext = [text['body_text'][entry]['text'] for entry in range(len(text['body_text']))]

    full_text = title_meta + text_abstract + text_bodytext
    return full_text

final_text = []
for i in tqdm(range(len(filenames))):
    file_name = open(path+'/'+filenames[i])
    text = json.loads(file_name.read())
    final_text.append(extract_text(text))

In [ ]:
final_str_text = [' '.join(final_text[i]) for i in range(len(final_text))]

Pre-processing steps

*   Lower casing
*   Remove Stop words

*   Remove punctuations like !()-[]{};:'"\,<>/?@#$%^&*_~
*   Remove website URLs
*   Remove non-alphanumeric characters i.e. everything except 
*   Remove numbers[0-9], letters and _


*   Remove strings which look like dates for e.g. **12/23/2016** and times **12:34:56**. Also remove strings which also are **am** or **pm** which are probably part of the preceding time in the source list
*   Lemmatization




In [ ]:
def preprocessing(data):
    text_lower = data.lower()

    STOPWORDS = set(stopwords.words('english'))
    text_stop = " ".join([word for word in text_lower.split() if word not in STOPWORDS])

    text_punc = re.sub(r'[^\w\s^.]', '', text_stop)
    text_url = re.sub(r'https?://\S+|www\.\S+', '', text_punc)

    text_date = re.sub(r'^(?:(?:[0-9]{2}[:\/,]){2}[0-9]{2,4}|am|pm)$', '', text_url)
    
    # removing non-alphanumeric characters
    text_alnum = re.sub(r'\W+^.', ' ', text_date)

    # removing numbers (0-9)
    text_number = re.sub(r'[0-9]+', '', text_alnum)
    
    wordnet_lemmatizer = WordNetLemmatizer()
    text_lemma = " ".join([wordnet_lemmatizer.lemmatize(word) for word in text_number.split(" ")])

    return text_lemma

In [4]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\surta\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
with open('preprocessed_final_with_dot.txt', 'w', encoding = 'utf8') as f:
    for item in tqdm(range(len(final_str_text))):
        if item != '':
            f.write("%s\n" % preprocessing(final_str_text[item]))

In [4]:
with open('C:/Users/surta/Downloads/preprocessed_final_with_dot.txt', 'r', encoding="utf8") as f:
    pre_text = f.readlines()

We have considered first 20 documents for this task.

In [5]:
pre_text = pre_text[:20]

We extract all the sentences from each document based on delimeter "." i.e. fullstop. 

In [6]:
sentences = [j.strip() for i in range(len(pre_text)) for j in pre_text[i].split(".") if j.strip() != '']

There are a total of 3002 sentences in the first 20 documents considered.

In [7]:
len(sentences)

3002

In [8]:
def compute_word_vectors(path): 
    '''
    computes the word vectors corresponding to the text corpus passed and returns the trained model
    '''
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    text = word2vec.Text8Corpus(path)
    model = word2vec.Word2Vec(text, vector_size = 128, window = 5, min_count = 50, workers=10, epochs = 1 )
    model.save('model1.bin')
    return model

In [ ]:
path = 'C:/Users/surta/Downloads/preprocessed_final_with_dot.txt'

word2vec = compute_word_vectors(path)
word2vec.save(root+'model_NLP_word2vec.bin')

In [9]:
word2vec = word2vec.Word2Vec.load('C:/Users/surta/Downloads/model_NLP.bin')

In [11]:
vocab = list(word2vec.wv.index_to_key)

In [12]:
embedding={}
for word in vocab:
    embedding[word] = word2vec.wv.get_vector(word)

In [13]:
path = Path('C:/Users/surta/Downloads/')
X = create_empty_array((len(sentences), 192*2), fname='X_on_disk', path=path, mode='r+')

In [27]:
def compute_sent_emb(embedding, sentences, nhid, dir = 1):
    h0 = np.random.normal(0, 0.01, nhid)
    U = np.random.normal(0, 0.01, (nhid, nhid))
    W = np.random.normal(0,0.01,(nhid, 128))
    prev_h_forward = U.dot(h0)
    prev_h_back = U.dot(h0)
    h_forward = np.zeros(nhid)
    h_back = np.zeros(nhid)
    i = 0
    for sent in tqdm(sentences):
        if dir in [0,1]:
            for word in sent.split():
                if word in embedding:
                    x = embedding[word]
                    y = stats.zscore(x)
                    h_forward = np.tanh(np.dot(W, y) + np.dot(U, prev_h_forward))
                    prev_h_forward = h_forward/nhid
        if dir in [0,2]:
            for word in sent.split()[::-1]:
                if word in embedding:
                    x = embedding[word]
                    y = stats.zscore(x)
                    h_back = np.tanh(np.dot(W, y) + np.dot(U, prev_h_back))
                    prev_h_back = h_back/nhid
        X[i] = np.concatenate((h_forward, h_back))
        i += 1

In [28]:
compute_sent_emb(embedding, sentences, nhid = 192)

100%|█████████████████████████████████████████████████████████████████████████████| 3002/3002 [00:03<00:00, 755.15it/s]


In [29]:
def similar_sent_finder(index):
    lst1=[]
    sim=np.zeros(len(sentences))
    k = index
    i=0
    for l in range(len(sentences)):
        p = fastdist.cosine(X[k],X[l])
        sim[i]=p
        i=i+1
    ind = np.argpartition(sim, -10)[-10:]
    for i in ind:
        lst1.append((sentences[i],sim[i]))
    sim_df = pd.DataFrame(lst1)
    sim_df.columns = ['Sentence', 'Cosine Similarity']
    sim_df = sim_df.sort_values(by = ['Cosine Similarity'], ascending = False)
    return sim_df.iloc[1:]

In [30]:
sent1 = similar_sent_finder(0)
print("Example 1:" , sentences[0])
sent1

Example 1: multifaceted covid russian experience according current live statistic time editing letter russia third country world affected covid new case death rate rising


,Sentence,Cosine Similarity
8,low temperature between c c small peptide concentration dependence clear temperature correlation whereas high temperature between c c strong decrease hwhm presence inclusion depending pl content without variation temperature rise,0.666299
4,tested antibody following strain titre assigned value order allow calculation titre rise,0.666281
7,asymptomatic atypical presentation covid continue present ed number covid case rise,0.666281
2,doctor team reported appreciating ot available help cognitive assessment especially mental capacity concern treatment declined,0.649984
6,conclusion year study single medical institution found use teicoplanin linezolid significantly increased,0.615510
1,number annual inpatientdays hospital increased,0.615462
3,addition consumption tigecycline increased,0.615431
5,overall plasma concentration cell chemoattractants namely cxc chemokine mig cc chemokine rantes significantly elevated,0.531741
0,contrast hn infection plasma tnfa elevated patients,0.531737


In [22]:
sent2 = similar_sent_finder(5)
print("Example 2:" , sentences[5])
sent2

Example 2: staff undergone online hour training course become qualified coronavirus infection treatment


,Sentence,Cosine Similarity
4,rsv virion contains two surface glycoprotein f g vital initial phase infection making critical target rsv treatment,1.0
7,apart chemokine il also observed elevation monocytemacrophages b chemokine mcp soon onset sars followed decline corticosteroid treatment,1.0
6,sarscovinfected patient also showed elevation plasma il soon disease onset declined day corticosteroid treatment,1.0
9,initiated study comparing plasma cytokine chemokines sars patient without corticosteroid treatment,1.0
8,measured cytokine chemokines il b il il ilp tnfa ifng rantes mig show significant difference corticosteroid treatment,1.0
5,lipid based system exhibited safe treatment,1.0
2,found rrtpcr result several test different time point variable patient course diagnosis treatment,1.0
1,vns started day ds treatment,1.0
0,significant possibility hn subtype influenza source next pandemic influenza strain pathophysiology clinical course hn influenza virus infection identification key objective clinical data crucial piece information help physician provide timely adequate treatment,1.0


In [23]:
sent3 = similar_sent_finder(10)
print("Example 3:" , sentences[10])
sent3

Example 3: teaching training mostly translated via online platform excluded opportunity get clinical experience surgery


,Sentence,Cosine Similarity
7,surgeon covid dedicated hospital rarely practice surgery,1.000000
6,icu patient need mechanical ventilation percutaneous tracheostomy endoscopic control mostly performed decrease aerosol formation viral load staff complication associated endotracheal tube comparison surgical tracheostomy,0.634440
8,surgery benign disorder precancerous lesion reconstructive procedure currently postponed,0.602287
5,plasmodium falciparum line d pfswib pfswib first thawed established continuous cultivation vitro according standard procedure,0.584318
2,question designed seek one response respondent suggesting preference total percentage category combined one hundred percent various organization difficulty convincing decision maker invest emergency management nonemergency periods,0.582420
9,procedure performed mouse anesthesia,0.573576
4,quickly became clear unmet need infection control support related pandemic team contacted several care home rehabilitation facilities,0.548207
1,nanoparticles show great potential biomedical application especially patient relapse completing conventional antiviral therapy,0.543088
0,fatality rate associated pediatric acute respiratory distress syndrome ards decreased recent decade advance medical treatment especially respiratory management lungprotective therapy,0.543003


In [24]:
sent4 = similar_sent_finder(15)
print("Example 4:" , sentences[15])
sent4

Example 4: patient diagnosed coronavirus infection treated home admitted covid dedicated multidisciplinary hospital surgical care provided


,Sentence,Cosine Similarity
6,obtained,0.617160
4,falciparum d obtained plasmodb httpplasm odb,0.617157
5,protocol approved clinical research ethic committee chinese university hong kong informed consent obtained participants,0.617156
2,informed written consent obtained patients,0.617156
1,patient consent publication obtained,0.617109
9,two stable transfections pfswibfkbplid integrated parasite clone pfswibc pfswibb obtained,0.617082
7,patient sample basic clinical data available,0.543192
8,personal protective equipment including highvis jacket reinforced toe shoe hard hat available,0.543185
0,major challenge combatting infectious disease sars mers covid including fact effective drug vaccine available,0.543115


In [25]:
sent5 = similar_sent_finder(20)
print("Example 5:" , sentences[20])
sent5

Example 5: moreover observational department organized surgical hospital patient allocation coronavirus infection status defined


,Sentence,Cosine Similarity
7,study fipv positive cell isolated pyogranulomas exudate naturally fipvinfected cat presence two immunologic target viral antigen mhc i surface determined,0.583592
6,fold change experimental reference group calculated δδcq,0.568962
9,two form distinguished,0.546197
5,consequence negative rrtpcr lead false assumption patient infected sarscov clinical parameter need considered,0.538020
4,protein expression level might importance context assessed,0.527675
2,subsequently coexpression ace tmprss nachrs assessed,0.527632
3,bead incubated l plasma different cytokine chemokines sample captured corresponding beads,0.525418
1,phenomenon largely due role medical profession assumed,0.481340
0,fipv positive cell mhc present showing internalization retention mhc molecule occurs,0.471919
